# Artificial Neural Network

### Importing the libraries

In [1]:
import numpy as np
import pandas as pd
import tensorflow as  tf

In [2]:
tf.__version__

'2.10.0'

## Part 1 - Data Preprocessing

### Importing the dataset

In [65]:
dataset = pd.read_csv('Churn_Modelling.csv')
X = dataset.iloc[: ,3:-1].values   #the first 3 rows are not impact on descision.
y = dataset.iloc[: , -1].values

In [66]:
print(X)
print(y)

[[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]]
[1 0 1 ... 1 1 0]


### Encoding categorical data

Label Encoding the "Gender" column

In [68]:
# categorical features need tobe  encoded to Numerical (3rd col ->gender)
from sklearn.preprocessing import LabelEncoder
le  = LabelEncoder()            #labelEncoding -> covert gender(categorical) to male(binary 0: female, 1: male) , and no female column.
                                # if female(binary) col present , then it is OneHotEncoding
X[:, 2] = le.fit_transform(X[:, 2])


In [69]:
print(X)

[[619 'France' 0 ... 1 1 101348.88]
 [608 'Spain' 0 ... 0 1 112542.58]
 [502 'France' 0 ... 1 0 113931.57]
 ...
 [709 'France' 0 ... 0 1 42085.58]
 [772 'Germany' 1 ... 1 0 92888.52]
 [792 'France' 0 ... 1 0 38190.78]]


One Hot Encoding the "Geography" column

In [70]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
X = np.array(ct.fit_transform(X))

# the 3 new columns ,for replace of geography, appear at beging of table

In [71]:
print(X)

[[1.0 0.0 0.0 ... 1 1 101348.88]
 [0.0 0.0 1.0 ... 0 1 112542.58]
 [1.0 0.0 0.0 ... 1 0 113931.57]
 ...
 [1.0 0.0 0.0 ... 0 1 42085.58]
 [0.0 1.0 0.0 ... 1 0 92888.52]
 [1.0 0.0 0.0 ... 1 0 38190.78]]


### Splitting the dataset into the Training set and Test set

In [72]:
from sklearn.model_selection import train_test_split
X_train, X_test,y_train , y_test = train_test_split(X, y, test_size=0.2,random_state=0)

### Feature Scaling

In [73]:
# some features have higher numerical values. so scaling of them is required.
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

## Part 2 - Building the ANN

### Initializing the ANN

In [74]:
ann = tf.keras.models.Sequential()

### Adding the input layer and the first hidden layer

In [75]:
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))   #add allows to add hidden  layer to ann model
# relu = rectifier activation function
# unit = 6 means there are 6 input nodes in input layer

### Adding the second hidden layer

In [76]:
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

### Adding the output layer

In [77]:
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid')) #ouput  is only one variable, so only 1 node

# here binaryclassification  need to be used so -> sigmoid
# if categorical classification then ->'soft max'

## Part 3 - Training the ANN

### Compiling the ANN

In [78]:
ann.compile(optimizer= 'adam', loss= 'binary_crossentropy', metrics=  ['accuracy'])

### Training the ANN on the Training set

In [79]:
ann.fit(X_train, y_train , batch_size=32,  epochs= 100)  #epochs helps in error correction

#divisions size - 32 .eposch = number of time to  backtrack 

Epoch 1/100
250/250 [==============================] - 1s 956us/step - loss: 0.7257 - accuracy: 0.5584
Epoch 2/100
250/250 [==============================] - 0s 964us/step - loss: 0.5286 - accuracy: 0.7959
Epoch 3/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4632 - accuracy: 0.7977
Epoch 4/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4300 - accuracy: 0.8139
Epoch 5/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4103 - accuracy: 0.8250
Epoch 6/100
250/250 [==============================] - 0s 996us/step - loss: 0.3944 - accuracy: 0.8342
Epoch 7/100
250/250 [==============================] - 0s 984us/step - loss: 0.3821 - accuracy: 0.8422
Epoch 8/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3728 - accuracy: 0.8460
Epoch 9/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3661 - accuracy: 0.8490
Epoch 10/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3614 

## Part 4 - Making the predictions and evaluating the model

### Predicting the result of a single observation

In [84]:
# input shouldbe 2d array
# required features should be encoded and scaled before use
# 1, 0, 0 for france


print(ann.predict(sc.transform([[ 1, 0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]])) > 0.5)

# ann.predict gives the probality value of (non)leaving the bank by customer

1/1 [==============================] - 0s 17ms/step
[[False]]


using ANN model to predict if the customer with the following informations will leave the bank: 

Geography: France

Credit Score: 600

Gender: Male

Age: 40 years old

Tenure: 3 years

Balance: \$ 60000

Number of Products: 2

Does this customer have a credit card? Yes

Is this customer an Active Member: Yes

Estimated Salary: \$ 50000

Will the customer leave the bank ?

**Solution**

In [81]:
print(ann.predict(sc.transform([[1, 0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]])) > 0.5)

1/1 [==============================] - 0s 27ms/step
[[False]]


### Predicting the Test set results

In [85]:
y_pred = ann.predict(X_test)
y_pred = (y_pred > 0.5 )
print(np.concatenate((y_pred.reshape(len(y_pred), 1), y_test.reshape(len(y_test), 1)), 1))


63/63 [==============================] - 0s 893us/step
[[0 0]
 [0 1]
 [0 0]
 ...
 [0 0]
 [0 0]
 [0 0]]


### Making the Confusion Matrix

In [83]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)  # y_true, y_pred
print(cm)
accuracy_score(y_test, y_pred)

[[1518   77]
 [ 208  197]]


0.8575